In [1]:
import databricks.koalas as ks
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[6]").setAppName('ad eda')
sc = SparkContext(conf=conf)

In [ ]:
def get_train_data():
    '''
    合并训练数据集
    '''
    train_user = ks.read_csv("data/train_preliminary/user.csv")

    train_click_log = ks.read_csv("data/train_preliminary/click_log.csv")

    train_ad = ks.read_csv("data/train_preliminary/ad.csv")
    train_data = train_user.merge(train_click_log, on="user_id",
                                  how='inner').merge(train_ad,
                                                     on="creative_id",
                                                     how='inner')
    return train_data


def get_test_data():
    '''
    合并测试集
    '''
    test_click_log = ks.read_csv("data/test/click_log.csv")
    test_ad = ks.read_csv("data/test/ad.csv")
    test_data = test_click_log.merge(test_ad, on="creative_id")
    return test_data

train_data = get_train_data()
train_data = train_data.drop(['age', 'gender'], axis=1)
test_data = get_test_data()
corpus = ks.concat([train_data, test_data], axis=0)
query = '''
select
   `time`,
   creative_id,
   click_times,
   ad_id,
   product_id,
   product_category,
   advertiser_id,
   industry,
   row_number()
           over (partition by b.time,click_times,product_id, product_category,advertiser_id,industry order by 1 desc) ad_rn
from (select `time`,
         creative_id,
         click_times,
         ad_id,
         product_id,
         product_category,
         advertiser_id,
         industry
  from {corpus}
  group by `time`,
         creative_id,
         click_times,
         ad_id,
         product_id,
         product_category,
         advertiser_id,
         industry
 ) b
'''
encode_result = ks.sql(query=query, corpus=corpus)
print(encode_result.nunique())